In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import wrangle
import f_engineer
import warnings
warnings.filterwarnings("ignore")

import sklearn.preprocessing as skl_pp

from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector, f_classif

from sklearn.linear_model import LinearRegression

import pydataset as pds

1. Load the tips dataset.

- Create a column named price_per_person. This should be the total bill divided by the party size.
- Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?
- Use select k best to select the top 2 features for predicting tip amount. What are they?
- Use recursive feature elimination to select the top 2 features for tip amount. What are they?
- Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

In [2]:
# load the 'tips' dataset
df = sns.load_dataset('tips')

In [3]:
# # Create a column named price_per_person. This should be the total bill divided by the party size.
df['price_per_person'] = df['total_bill'] / df['size']
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person
0,16.99,1.01,Female,No,Sun,Dinner,2,8.495000
1,10.34,1.66,Male,No,Sun,Dinner,3,3.446667
2,21.01,3.50,Male,No,Sun,Dinner,3,7.003333
3,23.68,3.31,Male,No,Sun,Dinner,2,11.840000
4,24.59,3.61,Female,No,Sun,Dinner,4,6.147500


In [4]:
# convert smoker into is_smoker 1 or 0
df['is_smoker'] = df.smoker.isin(['Yes']).astype(int)
# convert sex into boolean 1 = male, 0 = dinner
df['is_male'] = df.sex.isin(['Male']).astype(int)
# create dummy columns for day
dummy_df = pd.get_dummies(df[['day']], dummy_na=False, drop_first=[True])
df = pd.concat([df, dummy_df], axis=1)
# create a new column named day_Thur if all other days are 0 than it is 1
df['day_Thur'] = (df.day_Fri + df.day_Sat + df.day_Sun == 0).astype(int)
# drop the columns we don't need
df.drop(['day', 'time', 'sex', 'smoker'], axis=1, inplace=True)
df.head()

,total_bill,tip,size,price_per_person,is_smoker,is_male,day_Fri,day_Sat,day_Sun,day_Thur
0,16.99,1.01,2,8.495000,0,0,0,0,1,0
1,10.34,1.66,3,3.446667,0,1,0,0,1,0
2,21.01,3.50,3,7.003333,0,1,0,0,1,0
3,23.68,3.31,2,11.840000,0,1,0,0,1,0
4,24.59,3.61,4,6.147500,0,0,0,0,1,0


In [5]:
# turned code above into a function
def prep_tips(df):
    df['price_per_person'] = df['total_bill'] / df['size']
    # convert smoker into is_smoker 1 or 0
    df['is_smoker'] = df.smoker.isin(['Yes']).astype(int)
    # convert sex into boolean 1 = male, 0 = dinner
    df['is_male'] = df.sex.isin(['Male']).astype(int)
    # create dummy columns for day
    dummy_df = pd.get_dummies(df[['day']], dummy_na=False, drop_first=[True])
    df = pd.concat([df, dummy_df], axis=1)
    # create a new column named day_Thur if all other days are 0 than it is 1
    df['day_Thur'] = (df.day_Fri + df.day_Sat + df.day_Sun == 0).astype(int)
    # drop the columns we don't need
    df.drop(['day', 'time', 'sex', 'smoker'], axis=1, inplace=True)
    return df

Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?

- The most important features I chose to determinr the tip amount are:
    - total_bill
    - size of the party
    - price per person


In [6]:
# split the data into training and test sets
train, validate, test = wrangle.split_dataframe(df)
# seperate the target variable from the features
y_train = train.tip
X_train = train.drop('tip', axis=1)
y_validate = validate.tip
X_validate = validate.drop('tip', axis=1)
y_test = test.tip
X_test = test.drop('tip', axis=1)

print(train.shape)
print(validate.shape)
print(test.shape)

(136, 10)
(59, 10)
(49, 10)


In [7]:
# scale the data 
scaler = skl_pp.MinMaxScaler()
# Fit it
scaler.fit(X_train)
# Use it
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

In [8]:
# create a function that scales useing MinMaxScaler
def scale_minmax(df):
    scaler = skl_pp.MinMaxScaler()
    scaler.fit(df)
    df_scaled = scaler.transform(df)
    df_scaled = pd.DataFrame(df_scaled, columns=df.columns)
    return df_scaled


In [9]:
# convert the data into a dataframe
X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_train_df

,total_bill,size,price_per_person,is_smoker,is_male,day_Fri,day_Sat,day_Sun,day_Thur
0,0.210561,0.2,0.187736,1.0,1.0,0.0,1.0,0.0,0.0
1,0.188688,0.2,0.158965,0.0,1.0,0.0,1.0,0.0,0.0
2,0.407203,0.6,0.133973,0.0,1.0,0.0,0.0,1.0,0.0
3,0.284578,0.2,0.285092,0.0,1.0,0.0,0.0,0.0,1.0
4,0.291648,0.2,0.294391,1.0,0.0,1.0,0.0,0.0,0.0
5,0.307114,0.4,0.150344,0.0,0.0,0.0,0.0,1.0,0.0
6,0.175652,0.2,0.141819,1.0,1.0,0.0,1.0,0.0,0.0
7,0.532700,0.2,0.611450,1.0,0.0,0.0,1.0,0.0,0.0
8,0.362793,0.2,0.387969,0.0,1.0,0.0,0.0,1.0,0.0
9,0.560539,0.2,0.648067,1.0,1.0,0.0,0.0,0.0,1.0


Use select k best to select the top 2 features for predicting tip amount. What are they?


In [10]:
# Use select k best to select the top 2 features for predicting tip amount.
kbest = SelectKBest(f_regression, k=2)
# kbest_class = SelectKBest(f_classif, k=2)
kbest.fit(X_train_df, y_train)
# kbest_class.fit(X_train_c, y_train)
kbest_results_num = pd.DataFrame(dict(p=kbest.pvalues_, f=kbest.scores_), index=X_train_df.columns)
kbest_results_num

,p,f
total_bill,5.022341e-17,92.909255
size,6.609409e-11,50.406244
price_per_person,2.103984e-03,9.835918
is_smoker,4.162147e-01,0.665093
is_male,9.994374e-02,2.744264
day_Fri,4.022814e-01,0.705970
day_Sat,4.951505e-01,0.467869
day_Sun,2.398630e-01,1.393757
day_Thur,1.338335e-01,2.274950


In [11]:
X_train_df.columns[kbest.get_support()]

Index(['total_bill', 'size'], dtype='object')

In [12]:
X_train_transformed = pd.DataFrame(
    kbest.transform(X_train_df),
    index=X_train_df.index,
    columns=X_train_df.columns[kbest.get_support()]
)
X_train_transformed.head()

,total_bill,size
0,0.210561,0.2
1,0.188688,0.2
2,0.407203,0.6
3,0.284578,0.2
4,0.291648,0.2


Use recursive feature elimination to select the top 2 features for tip amount. What are they?


In [13]:
# Use recursive feature elimination to select the top 2 features for tip amount.
model = LinearRegression()
rfe = RFE(model, n_features_to_select=2)
rfe.fit(X_train_df, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [14]:
pd.DataFrame({'rfe_ranking': rfe.ranking_}, index=X_train_df.columns)

,rfe_ranking
total_bill,1
size,7
price_per_person,1
is_smoker,8
is_male,6
day_Fri,2
day_Sat,5
day_Sun,3
day_Thur,4


In [15]:
X_train_df.columns[rfe.get_support()]

Index(['total_bill', 'price_per_person'], dtype='object')

In [16]:
X_train_transformed = pd.DataFrame(
    rfe.transform(X_train_scaled),
    index=X_train_df.index,
    columns=X_train_df.columns[rfe.support_]
)
X_train_transformed.head()

,total_bill,price_per_person
0,0.210561,0.187736
1,0.188688,0.158965
2,0.407203,0.133973
3,0.284578,0.285092
4,0.291648,0.294391


Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [17]:
# Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class.
def select_kbest(X, y, k):
    kbest = SelectKBest(f_regression, k=k)
    kbest.fit(X, y)
    return X.columns[kbest.get_support()]

In [18]:
#  Test your function with the tips dataset. 
# You should see the same results as when you did the process manually.
select_kbest(X_train_df, y_train, 2)

Index(['total_bill', 'size'], dtype='object')

3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [19]:
# Write a function named rfe that takes in the predictors, the target, and the number of features to select.
def rfe(X, y, k):
    model = LinearRegression()
    rfe = RFE(model, n_features_to_select=k)
    rfe.fit(X, y)
    feature_mask = rfe.support_
    rfe_feature = X.iloc[:,feature_mask].columns.tolist()
    var_ranks = rfe.ranking_
    # get the variable names
    var_names = X.columns.tolist()
    # combine ranks and names into a df for clean viewing
    rfe_ranks_df = pd.DataFrame({'Var': var_names, 'Rank': var_ranks})
    return rfe_feature, rfe_ranks_df.sort_values('Rank')

In [20]:
# Test your function with the tips dataset. You should see the same results as when you did the process manually.
rfe(X_train_df, y_train, 3)

(['total_bill', 'price_per_person', 'day_Fri'],
                 Var  Rank
 0        total_bill     1
 2  price_per_person     1
 5           day_Fri     1
 7           day_Sun     2
 8          day_Thur     3
 6           day_Sat     4
 4           is_male     5
 1              size     6
 3         is_smoker     7)

4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [21]:
# Load the swiss dataset from pydataset and use all the other features to predict Fertility.
swiss = pds.data('swiss')
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [22]:
# split the swiss
train, validate, test = wrangle.split_dataframe(swiss)


In [23]:
# select only the name of the feature columns not the target column
y = swiss['Fertility']
f_engineer.scale_data(train, validate, test, y)
X_train_df
y_train

KeyError: "None of [Float64Index([80.2, 83.1, 92.5, 85.8, 76.9, 76.1, 83.8, 92.4, 82.4, 82.9, 87.1,\n              64.1, 66.9, 68.9, 61.7, 68.3, 71.7, 55.7, 54.3, 65.1, 65.5, 65.0,\n              56.6, 57.4, 72.5, 74.2, 72.0, 60.5, 58.3, 65.4, 75.5, 69.3, 77.3,\n              70.5, 79.4, 65.0, 92.2, 79.3, 70.4, 65.7, 72.7, 64.4, 77.6, 67.6,\n              35.0, 44.7, 42.8],\n             dtype='float64')] are in the [columns]"

In [ ]:
# # scale the data
# scaler = skl_pp.MinMaxScaler()
# # Fit it
# scaler.fit(X_train)
# # Use it
# X_train_scaled = scaler.transform(X_train)
# X_validate_scaled = scaler.transform(X_validate)
# X_test_scaled = scaler.transform(X_test)
# # convert the data into a dataframe
# X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
# X_train_df.head()

In [ ]:
# utilize kbest function
select_kbest(X_train_df, y_train, 3)

In [ ]:
# utilize rfe function
rfe(X_train_df, y_train, 3)